### Clone keras yolov3 repository

In [1]:
%cd /data1
!git clone https://github.com/qqwweee/keras-yolo3.git keras_yolo3

/data1
Cloning into 'keras_yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 836.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


### Load and convert yolov3 model to keras

In [2]:
%cd keras_yolo3
!wget https://pjreddie.com/media/files/yolov3.weights
!python convert.py yolov3.cfg yolov3.weights /data1/yolo.h5

/data1/keras_yolo3
--2021-08-23 11:49:28--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  7.56MB/s    in 31s     

2021-08-23 11:50:00 (7.62 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

/workspace/miniconda/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]

### Download kneron images for further quantization

In [3]:
%cd /data1
!wget http://doc.kneron.com/docs/toolchain/res/test_image10.zip
!unzip test_image10.zip

/data1
--2021-08-23 11:51:19--  http://doc.kneron.com/docs/toolchain/res/test_image10.zip
Resolving doc.kneron.com (doc.kneron.com)... 75.26.5.105
Connecting to doc.kneron.com (doc.kneron.com)|75.26.5.105|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2615052 (2.5M) [application/zip]
Saving to: ‘test_image10.zip’

test_image10.zip    100%[===================>]   2.49M  1.46MB/s    in 1.7s    

2021-08-23 11:51:21 (1.46 MB/s) - ‘test_image10.zip’ saved [2615052/2615052]

Archive:  test_image10.zip
   creating: test_image10/
 extracting: test_image10/000000000139.jpg  
  inflating: test_image10/000000000785.jpg  
  inflating: test_image10/000000000872.jpg  
  inflating: test_image10/000000000885.jpg  
 extracting: test_image10/000000001000.jpg  
  inflating: test_image10/000000001268.jpg  
  inflating: test_image10/000000001296.jpg  
  inflating: test_image10/000000005001.jpg  
  inflating: test_image10/000000005193.jpg  
  inflating: test_image10/309_190.jpg  


### Copy image for model's testing in different convertation stages

In [4]:
%cd /data1
!cp /workspace/E2E_Simulator/app/test_image_folder/yolo/000000350003.jpg ./.

/data1


### Write and run example script test.py to convert yolov3 from keras to nef

In [5]:
%%writefile test.py

import ktc
import os
import onnx
from PIL import Image
import numpy as np

import tensorflow as tf
import pathlib
import sys
sys.path.append(str(pathlib.Path("keras_yolo3").resolve()))
from yolo3.model import yolo_eval

# Postprocess function of Yolov3
def postprocess(inf_results, ori_image_shape):
    tensor_data = [tf.convert_to_tensor(data, dtype=tf.float32) for data in inf_results]

    # Get anchor info
    anchors_path = "/data1/keras_yolo3/model_data/yolo_anchors.txt" 
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    anchors = np.array(anchors).reshape(-1, 2)

    # Postprocess
    num_classes = 80
    boxes, scores, classes = yolo_eval(tensor_data, anchors, num_classes, ori_image_shape)
    with tf.Session() as sess:
        boxes = boxes.eval()
        scores = scores.eval()
        classes = classes.eval()

    return boxes, scores, classes

from yolo3.utils import letterbox_image

# Preprocess function of Yolov3 
def preprocess(pil_img):
    model_input_size = (416, 416)  # to match our model input size when converting
    boxed_image = letterbox_image(pil_img, model_input_size)
    np_data = np.array(boxed_image, dtype='float32')

    np_data /= 255.
    return np_data

# Convert h5 model to onnx
m = ktc.onnx_optimizer.keras2onnx_flow("/data1/yolo.h5", input_shape = [1,416,416,3])
m = ktc.onnx_optimizer.onnx2onnx_flow(m)
onnx.save(m,'yolo.opt.onnx')


# Npu(only) performance simulation
km = ktc.ModelConfig(1001, "0001", "520", onnx_model=m)
eval_result = km.evaluate()
print("\nNpu performance evaluation result:\n" + str(eval_result))



# Onnx model check
input_image = Image.open('/data1/000000350003.jpg')
in_data = preprocess(input_image)
out_data = ktc.kneron_inference([in_data], onnx_file="/data1/yolo.opt.onnx", input_names=["input_1_o0"])
det_res = postprocess(out_data, [input_image.size[1], input_image.size[0]])
print(det_res)

# Load and normalize all image data from folder
img_list = []
for (dir_path, _, file_names) in os.walk("/data1/test_image10"):
    for f_n in file_names:
        fullpath = os.path.join(dir_path, f_n)
        print("processing image: " + fullpath)

        image = Image.open(fullpath)
        img_data = preprocess(image)
        img_list.append(img_data)


# Fix point analysis
bie_model_path = km.analysis({"input_1_o0": img_list})
print("\nFix point analysis done. Save bie model to '" + str(bie_model_path) + "'")


# Bie model check
input_image = Image.open('/data1/000000350003.jpg')
in_data = preprocess(input_image)
out_data = ktc.kneron_inference([in_data], bie_file=bie_model_path, input_names=["input_1_o0"])
det_res = postprocess(out_data, [input_image.size[1], input_image.size[0]])
print(det_res)


# Compile
nef_model_path = ktc.compile([km])
print("\nCompile done. Save Nef file to '" + str(nef_model_path) + "'")

# Nef model check
input_image = Image.open('/data1/000000350003.jpg')
in_data = preprocess(input_image)
out_data = ktc.kneron_inference([in_data], nef_file=nef_model_path, radix=7)
det_res = postprocess(out_data, [input_image.size[1], input_image.size[0]])
print(det_res)

Writing test.py


In [6]:
!python test.py

/workspace/miniconda/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
Using TensorFlow backend.












/workspace/miniconda/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
Currently, custom input size is only available for single input size. Mystery node may generate wrong size.
running compiler and IP evaluator...
Compiler config generated.
Compilation and IP Evaluation finished.

Npu performance evaluation result:
***** Warning: this model has 1 CPU ops which may

### Make changes to E2E simulator in order to use it with multiprocessing

In [7]:
%%writefile /workspace/E2E_Simulator/python_flow/nef/nef.py
"""
Perform inference using an input NEF file.
"""
import ctypes
import math
import pathlib
import subprocess
import sys

import python_flow.common.directory_manager as dm
import python_flow.common.exceptions as exceptions
import python_flow.utils.utils as utils

FILE_DIR = pathlib.Path(__file__).resolve().parents[0]
E2E_DIR = pathlib.Path(__file__).resolve().parents[2]
NEF_LIB = ctypes.CDLL(FILE_DIR / "libnef.so")
CSIM520 = str(E2E_DIR / "python_flow/npu_sim")
CSIM720 = str(E2E_DIR / "python_flow/npu_csim")
BASE_INI = str(E2E_DIR / "python_flow/720.ini")

def advance(f, offset=0):
    f.read(offset)
    return int.from_bytes(f.read(4), byteorder='little')

def parse_fw_info(path, key):
    with open(path, 'rb') as f:
        model_cnt = advance(f)

        if "model_cnt" == key:
            yield model_cnt
            return

        model_info_list = []
        for _ in range(model_cnt):
            model_info = {}
            model_info['model_id'] = advance(f)
            model_info['model_ver'] = hex(advance(f))

            model_info['addr_in'] = hex(advance(f))
            model_info['size_in'] = hex(advance(f))

            model_info['addr_out'] = hex(advance(f))
            model_info['size_out'] = hex(advance(f))

            model_info['addr_wbuf'] = hex(advance(f))
            model_info['size_wbuf'] = hex(advance(f))

            model_info['addr_cmd'] = hex(advance(f))
            model_info['size_cmd'] = hex(advance(f))

            model_info['addr_wt'] = hex(advance(f))
            model_info['size_wt'] = hex(advance(f))

            model_info['addr_fw'] = hex(advance(f))
            model_info['size_fw'] = hex(advance(f))

            model_info_list.append(model_info)

            if "all" != key:
                yield model_info[key]

        if "all" == key:
            yield model_info_list
            #print(model_info_list)

def decompose_all_models(nef_file):
    """Separate the command/setup/weight binaries from the combined binary."""
    nef_folder = nef_file.parent
    models = nef_folder / "all_models.bin"
    fw_info = nef_folder / "fw_info.bin"
    model_cnt = next(parse_fw_info(fw_info, "model_cnt"))
    model_id_list = list(parse_fw_info(fw_info, "model_id"))
    size_cmd_list = list(parse_fw_info(fw_info, "size_cmd"))
    size_wt_list = list(parse_fw_info(fw_info, "size_wt"))
    size_fw_list = list(parse_fw_info(fw_info, "size_fw"))

    all_models = []
    with open(models, "r+b") as af:
        for idx in range(model_cnt):
            cur_model = "model_" + str(model_id_list[idx])
            all_models.append(cur_model)
            with open(nef_folder / (cur_model + "_command.bin"), "w+b") as cbf:
                size_cmd = int(size_cmd_list[idx], 16)
                cbf.write(af.read(size_cmd))
                cbf.close()
                pad = math.ceil(size_cmd / 16.0) * 16 - size_cmd
                af.read(pad) #pad to 16

            with open(nef_folder / (cur_model + "_weight.bin"), "w+b") as wbf:
                size_wt = int(size_wt_list[idx], 16)
                assert 0 == (size_wt % 16), "weight size should be aligned 16"
                wbf.write(af.read(size_wt))
                wbf.close()

            with open(nef_folder / (cur_model + "_setup.bin"), "w+b") as fbf:
                size_fw = int(size_fw_list[idx], 16)
                fbf.write(af.read(size_fw))
                fbf.close()
                pad = math.ceil(size_fw / 16.0) * 16 - size_fw
                af.read(pad) #pad to 16

    return all_models

def parse_nef(nef_file):
    """Get the combined binary, firmware info, and platform from the NEF file in C."""
    c_function = NEF_LIB.parse_nef
    c_function.argtypes = [ctypes.c_char_p]
    c_function.restype = ctypes.c_int
    platform = c_function(str(nef_file).encode())

    if platform == -1:
        raise exceptions.LibError(f"Could not parse NEF file correctly: {nef_file}")

    all_models = decompose_all_models(nef_file)
    return platform, all_models

def setup_nef(nef, model_id, model = '0'):
    """Parses NEF model and does extra checks and setup."""
    if not nef.exists():
        raise exceptions.InvalidInputError(f"Input NEF file does not exist: {nef}")

    platform, all_models = parse_nef(nef)
    platform = 520 if platform == 0 else 720

    model_num = "model_" + str(model_id) # potential model to run
    if len(all_models) == 1:
        model_num = all_models[0]
    elif model_num not in all_models:
        raise exceptions.InvalidInputError(f"Specified model ID not found in NEF. ID = {model_id}")
    input_folder = nef.parent / "out/nef/inputs" / model_num / model
    #print(input_folder)
    output_folder = nef.parent / "out/nef/outputs" / model_num / model
    input_folder.mkdir(parents=True, exist_ok=True)
    output_folder.mkdir(parents=True, exist_ok=True)

    return platform, model_num, input_folder, output_folder

def prep_csim_inputs(pre_results, input_folder, radix, platform):
    """Prepare the input files for NEF model."""
    input_files = []
    for index, result in enumerate(pre_results):
        file_name = "_".join([str(platform), "csim_rgba_in", str(index)]) + ".bin"
        input_file = str(input_folder / file_name)
        input_files.append(input_file)
        utils.convert_pre_numpy_to_rgba(result, input_file, radix, platform)

    return input_files

def update_ini(base_ini, new_ini, command, weight, setup, inputs):
    """Updates input INI file for the CSIM 720 model. Assume all files already exist."""
    file_inputs = ",".join(inputs)
    updated_lines = {
        "file_command": "".join(["file_command = ", command, "\n"]),
        "file_weight": "".join(["file_weight = ", weight, "\n"]),
        "file_setup": "".join(["file_setup = ", setup, "\n"]),
        "file_input": "".join(["file_input = ", file_inputs, "\n"])
    }

    new_file = []
    with open(base_ini, "r") as in_file:
        for line in in_file.readlines():
            need_update = None
            for key in updated_lines:
                if line.startswith(key):
                    need_update = key
                    break

            if need_update is not None:
                new_file.append(updated_lines[need_update])
                del updated_lines[need_update]
            else:
                new_file.append(line)

    with open(new_ini, "w") as out_file:
        out_file.write("".join(new_file))

def nef_inference(model_name, platform, data_type, nef_folder, input_files,
                  input_folder, output_folder, reordering, ioinfo_file, dump, threads = 16):
    """Performs inference on the specified model parsed from the NEF file."""
    #print(model_name, platform, data_type,"nef_folder: ", nef_folder,"input_files: ", input_files,
                  #"input_folder: ", input_folder,"output_folder", output_folder,"reordering: ", reordering,"ioinfo_file: ", ioinfo_file, "dump: ", dump)
    command = str(nef_folder / "_".join([model_name, "command.bin"]))
    setup = str(nef_folder / "_".join([model_name, "setup.bin"]))
    weight = str(nef_folder / "_".join([model_name, "weight.bin"]))
    with dm.DirectoryManager(output_folder):
        try:
            if platform == "520":
                subprocess.run([CSIM520, "-d", dump, command, weight, *input_files, setup,
                                "--thread", str(threads)], check=True)
                output = utils.csim_520_to_np(
                    str(output_folder), data_type, reordering, ioinfo_file, False)
            else:
                ini_file = input_folder / "720.ini"
                update_ini(BASE_INI, ini_file, command, weight, setup, input_files)
                subprocess.run([CSIM720, ini_file], check=True)
                output = utils.csim_720_to_np(
                    str(output_folder), data_type, reordering, ioinfo_file, False)
        except subprocess.CalledProcessError as error:
            raise exceptions.LibError(f"Hardware CSIM {platform} failed:\n{error}")
        except exceptions.ConfigError as error:
            sys.exit(error)
    return output

Overwriting /workspace/E2E_Simulator/python_flow/nef/nef.py


In [8]:
%%writefile /workspace/E2E_Simulator/python_flow/kneron_inference.py
"""
Generic inference function for ONNX, BIE, or NEF model.
"""
import pathlib
import sys

ROOT_FOLDER = pathlib.Path(__file__).resolve().parent.parent
sys.path.append(str(ROOT_FOLDER))

import python_flow.common.exceptions as exceptions
import python_flow.nef.nef as nef
import python_flow.dynasty.dynasty as dynasty

def kneron_inference(pre_results, nef_file="", onnx_file="", bie_file="", model_id=None,
                     input_names=[], radix=8, data_type="float", reordering=[],
                     ioinfo_file="", dump=False, platform=520, model='0', threads = 16):
    """Performs inference on the input model given the specified parameters.

    Input pre_results should be in format (1, h, w, c).

    Arguments:
        pre_results: List of NumPy arrays in channel last format from preprocessing
        nef_file: Path to NEF model for inference
        onnx_file: Path to ONNX model for inference, unused if nef_file is specified
        bie_file: Path to BIE model for inference, unused if nef_file/onnx_file is specified
        model_id: Integer of model to run inference, only necessary for NEF with multiple models
        input_names: List of input node names of BIE/ONNX model, unused if nef_file is specified
        radix: Integer radix to convert from float to fixed input
        data_type: String format of the resulting output, "fixed" or "float"
        dump: Boolean flag to dump intermediate nodes
        reordering: List of node names/integers specifying the output order
        ioinfo_file: String path to file mapping output node number to name, only used with NEF
        platform: Integer indicating platform of Dynasty fixed model
    """
    dump = 2 if dump else 0
    if nef_file:
        nef_path = pathlib.Path(nef_file).resolve()
        #print("nef_path :", nef_path)
        platform, model_name, input_folder, output_folder = nef.setup_nef(nef_path, model_id, model)
        #print("platform, model_name, input_folder, output_folder:", platform, model_name, input_folder, output_folder)
        input_files = nef.prep_csim_inputs(pre_results, input_folder, radix, str(platform))
        #print("input_files: ", input_files)
        output = nef.nef_inference(
            model_name, str(platform), data_type, nef_path.parent, input_files, input_folder,
            output_folder, reordering, ioinfo_file, str(dump), threads = 16)
    elif onnx_file:
        onnx = pathlib.Path(onnx_file).resolve()
        input_folder = onnx.parent / "out/onnx/inputs" / onnx.name
        output_folder = onnx.parent / "out/onnx/outputs" / onnx.name

        input_files = dynasty.prep_dynasty(
            pre_results, input_folder, output_folder, input_names, radix, platform, False)
        output = dynasty.dynasty_inference(
            onnx_file, "Float", str(platform), data_type, input_files, input_names,
            str(output_folder), reordering, dump)
    elif bie_file:
        bie = pathlib.Path(bie_file).resolve()
        input_folder = bie.parent / "out/bie/inputs" / bie.name
        output_folder = bie.parent / "out/bie/outputs" / bie.name

        input_files = dynasty.prep_dynasty(
            pre_results, input_folder, output_folder, input_names, radix, platform, True)
        output = dynasty.dynasty_inference(
            bie_file, "bie", str(platform), data_type, input_files, input_names,
            str(output_folder), reordering, dump)
    else:
        raise exceptions.RequiredConfigError("No input model selected for inference.")

    return output


Overwriting /workspace/E2E_Simulator/python_flow/kneron_inference.py


In [9]:
%%writefile /data1//keras_yolo3/yolo3/model.py
"""YOLO_v3 Model Defined in Keras."""

from functools import wraps

import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2

import sys
sys.path.append("/data1/keras_yolo3")
from yolo3.utils import compose


@wraps(Conv2D)
def DarknetConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)}
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    darknet_conv_kwargs.update(kwargs)
    return Conv2D(*args, **darknet_conv_kwargs)

def DarknetConv2D_BN_Leaky(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1))

def resblock_body(x, num_filters, num_blocks):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # Darknet uses left and top padding instead of 'same' mode
    x = ZeroPadding2D(((1,0),(1,0)))(x)
    x = DarknetConv2D_BN_Leaky(num_filters, (3,3), strides=(2,2))(x)
    for i in range(num_blocks):
        y = compose(
                DarknetConv2D_BN_Leaky(num_filters//2, (1,1)),
                DarknetConv2D_BN_Leaky(num_filters, (3,3)))(x)
        x = Add()([x,y])
    return x

def darknet_body(x):
    '''Darknent body having 52 Convolution2D layers'''
    x = DarknetConv2D_BN_Leaky(32, (3,3))(x)
    x = resblock_body(x, 64, 1)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    x = resblock_body(x, 512, 8)
    x = resblock_body(x, 1024, 4)
    return x

def make_last_layers(x, num_filters, out_filters):
    '''6 Conv2D_BN_Leaky layers followed by a Conv2D_linear layer'''
    x = compose(
            DarknetConv2D_BN_Leaky(num_filters, (1,1)),
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D_BN_Leaky(num_filters, (1,1)),
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D_BN_Leaky(num_filters, (1,1)))(x)
    y = compose(
            DarknetConv2D_BN_Leaky(num_filters*2, (3,3)),
            DarknetConv2D(out_filters, (1,1)))(x)
    return x, y


def yolo_body(inputs, num_anchors, num_classes):
    """Create YOLO_V3 model CNN body in Keras."""
    darknet = Model(inputs, darknet_body(inputs))
    x, y1 = make_last_layers(darknet.output, 512, num_anchors*(num_classes+5))

    x = compose(
            DarknetConv2D_BN_Leaky(256, (1,1)),
            UpSampling2D(2))(x)
    x = Concatenate()([x,darknet.layers[152].output])
    x, y2 = make_last_layers(x, 256, num_anchors*(num_classes+5))

    x = compose(
            DarknetConv2D_BN_Leaky(128, (1,1)),
            UpSampling2D(2))(x)
    x = Concatenate()([x,darknet.layers[92].output])
    x, y3 = make_last_layers(x, 128, num_anchors*(num_classes+5))

    return Model(inputs, [y1,y2,y3])

def tiny_yolo_body(inputs, num_anchors, num_classes):
    '''Create Tiny YOLO_v3 model CNN body in keras.'''
    x1 = compose(
            DarknetConv2D_BN_Leaky(16, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(32, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(64, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(128, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(256, (3,3)))(inputs)
    x2 = compose(
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(512, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'),
            DarknetConv2D_BN_Leaky(1024, (3,3)),
            DarknetConv2D_BN_Leaky(256, (1,1)))(x1)
    y1 = compose(
            DarknetConv2D_BN_Leaky(512, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))(x2)

    x2 = compose(
            DarknetConv2D_BN_Leaky(128, (1,1)),
            UpSampling2D(2))(x2)
    y2 = compose(
            Concatenate(),
            DarknetConv2D_BN_Leaky(256, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))([x2,x1])

    return Model(inputs, [y1,y2])


def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs


def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes


def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores


def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)

    return boxes_, scores_, classes_


def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format

    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer

    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value

    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true


def box_iou(b1, b2):
    '''Return iou tensor

    Parameters
    ----------
    b1: tensor, shape=(i1,...,iN, 4), xywh
    b2: tensor, shape=(j, 4), xywh

    Returns
    -------
    iou: tensor, shape=(i1,...,iN, j)

    '''

    # Expand dim to apply broadcasting.
    b1 = K.expand_dims(b1, -2)
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh/2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # Expand dim to apply broadcasting.
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh/2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou


def yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False):
    '''Return yolo_loss tensor

    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss

    Returns
    -------
    loss: tensor, shape=(1,)

    '''
    num_layers = len(anchors)//3 # default setting
    yolo_outputs = args[:num_layers]
    y_true = args[num_layers:]
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]
    input_shape = K.cast(K.shape(yolo_outputs[0])[1:3] * 32, K.dtype(y_true[0]))
    grid_shapes = [K.cast(K.shape(yolo_outputs[l])[1:3], K.dtype(y_true[0])) for l in range(num_layers)]
    loss = 0
    m = K.shape(yolo_outputs[0])[0] # batch size, tensor
    mf = K.cast(m, K.dtype(yolo_outputs[0]))

    for l in range(num_layers):
        object_mask = y_true[l][..., 4:5]
        true_class_probs = y_true[l][..., 5:]

        grid, raw_pred, pred_xy, pred_wh = yolo_head(yolo_outputs[l],
             anchors[anchor_mask[l]], num_classes, input_shape, calc_loss=True)
        pred_box = K.concatenate([pred_xy, pred_wh])

        # Darknet raw box to calculate loss.
        raw_true_xy = y_true[l][..., :2]*grid_shapes[l][::-1] - grid
        raw_true_wh = K.log(y_true[l][..., 2:4] / anchors[anchor_mask[l]] * input_shape[::-1])
        raw_true_wh = K.switch(object_mask, raw_true_wh, K.zeros_like(raw_true_wh)) # avoid log(0)=-inf
        box_loss_scale = 2 - y_true[l][...,2:3]*y_true[l][...,3:4]

        # Find ignore mask, iterate over each of batch.
        ignore_mask = tf.TensorArray(K.dtype(y_true[0]), size=1, dynamic_size=True)
        object_mask_bool = K.cast(object_mask, 'bool')
        def loop_body(b, ignore_mask):
            true_box = tf.boolean_mask(y_true[l][b,...,0:4], object_mask_bool[b,...,0])
            iou = box_iou(pred_box[b], true_box)
            best_iou = K.max(iou, axis=-1)
            ignore_mask = ignore_mask.write(b, K.cast(best_iou<ignore_thresh, K.dtype(true_box)))
            return b+1, ignore_mask
        _, ignore_mask = K.control_flow_ops.while_loop(lambda b,*args: b<m, loop_body, [0, ignore_mask])
        ignore_mask = ignore_mask.stack()
        ignore_mask = K.expand_dims(ignore_mask, -1)

        # K.binary_crossentropy is helpful to avoid exp overflow.
        xy_loss = object_mask * box_loss_scale * K.binary_crossentropy(raw_true_xy, raw_pred[...,0:2], from_logits=True)
        wh_loss = object_mask * box_loss_scale * 0.5 * K.square(raw_true_wh-raw_pred[...,2:4])
        confidence_loss = object_mask * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True)+ \
            (1-object_mask) * K.binary_crossentropy(object_mask, raw_pred[...,4:5], from_logits=True) * ignore_mask
        class_loss = object_mask * K.binary_crossentropy(true_class_probs, raw_pred[...,5:], from_logits=True)

        xy_loss = K.sum(xy_loss) / mf
        wh_loss = K.sum(wh_loss) / mf
        confidence_loss = K.sum(confidence_loss) / mf
        class_loss = K.sum(class_loss) / mf
        loss += xy_loss + wh_loss + confidence_loss + class_loss
        if print_loss:
            loss = tf.Print(loss, [loss, xy_loss, wh_loss, confidence_loss, class_loss, K.sum(ignore_mask)], message='loss: ')
    return loss


Overwriting /data1//keras_yolo3/yolo3/model.py


In [10]:
%%writefile /data1/parallel.py

import argparse
from os import listdir
from os.path import isfile, join
from time import time
from time import sleep

from keras_yolo3.yolo3.model import yolo_eval
from keras_yolo3.yolo3.utils import letterbox_image

import ktc
import tensorflow as tf
import onnx
from PIL import Image
import numpy as np


def postprocess(inf_results, ori_image_shape, conf_t  , iou_t ):
    tensor_data = [tf.convert_to_tensor(data, dtype=tf.float32) for data in inf_results]

    # Get anchor info
    anchors_path = "/data1/keras_yolo3/model_data/yolo_anchors.txt"
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    anchors = np.array(anchors).reshape(-1, 2)

    # Postprocess of Yolov3
    num_classes = 80
    boxes, scores, classes = yolo_eval(tensor_data, anchors, num_classes, ori_image_shape, score_threshold=conf_t, iou_threshold=iou_t)
    with tf.Session() as sess:
        boxes = boxes.eval()
        scores = scores.eval()
        classes = classes.eval()

    return boxes, scores, classes

# Preprocess of Yolov3
def preprocess(pil_img, img_size):    
    model_input_size = (img_size, img_size)  # to match our model input size when converting
    boxed_image = letterbox_image(pil_img, model_input_size)
    np_data = np.array(boxed_image, dtype='float32')

    np_data /= 255.
    return np_data


def setup_parser(test_args):
    """Setup the command line parser."""
    parser = argparse.ArgumentParser(description="Runs an inference on multiple images.",
                                     formatter_class=argparse.RawTextHelpFormatter)
    parser.add_argument("--demo",action="store_true", help="run demo on your image")
    parser.add_argument("--image", help="path to your image for demo", default="/data1/000000350003.jpg", type=str)
    parser.add_argument("--path", help="directory of your images", default="/workspace/COCO/valid2017", type=str)
    parser.add_argument("--nef", help="path to your nef model", default="/data1/batch_compile/models_520.nef", type=str)    
    parser.add_argument("--step", help="number of images for one model in every step", default=20, type=int)
    parser.add_argument("--init", help="initialization time between models", default=10, type=int)
    parser.add_argument("--model", help="model's number", default=0, type=int)
    parser.add_argument("--threads", help="choose number of workers for inference(only for 520 model)", default=16, type=int)
    parser.add_argument("--img-size", help="image size", default=416, type=int)
    parser.add_argument("--conf-t", help="confidence threshold", default=0.6, type=float) 
    parser.add_argument("--iou-t", help="iou threshold for NMS", default=0.5, type=float)
    #TODO add anchors
    return parser.parse_args(test_args)
        

def main(test_args = None):
    args = setup_parser(test_args)
    sleep(int(args.model)*args.init)
    step = args.step
    
    # Run demo on one image
    if args.demo:
        input_image = Image.open(args.image)
        print("Image is in: ", args.image)
        image = preprocess(input_image, args.img_size)
        
        tic = time()
        out_data = ktc.kneron_inference([image], nef_file=args.nef, radix=7 , model=str(args.model), threads=args.threads)
        print(f'inference time {(time()-tic):.2f}')

        # Postprocess
        tic = time()
        det_res = postprocess(out_data, [input_image.size[1], input_image.size[0]], args.conf_t  , args.iou_t)
        print(f'postprocess time {(time()-tic):.2f}')
        print(det_res)
        
    # Make prediction on multiple images with multiple models    
    else:
        images_folder = args.path + "/images"
        images = [f for f in listdir(images_folder) if isfile(join(images_folder, f))]
        predictions_folder = args.path + "/" + "predictions"
        images_folder = args.path + "/" + "images"
        in_data = []
        
        # Check intersection between predictions and images      
        predictions_name =[f[:12] for f in listdir(predictions_folder) if isfile(join(predictions_folder, f))]
        images_name =[f[:12] for f in listdir(images_folder) if isfile(join(images_folder, f))]
        predicted = list(set(predictions_name) & set(images_name))
        not_predicted = images_name

        # Remove already made predictions
        for element in predicted:                
            not_predicted.remove(element)

        # Create list with images without predictions    
        not_predicted_images = []
        for elem in not_predicted:
            not_predicted_images.append(elem + ".jpg")
            
        image_shape = [] # make list to fill with image sizes
        
        # Preprocess *step* images 
        for i in range(step):
            try:
                input_image = Image.open(args.path + "/images/" + not_predicted_images[i+args.model*step])
                pre = preprocess(input_image, args.img_size)
                image_shape.append(input_image.size)
                in_data.append(pre)
            except:
                print("No more images for model ", args.model)
                break               

        images = not_predicted_images
                   
        print("-"*10)
        print("Preprocessing is done!")
        print("-"*10)
        
        count = 0 # to assign image to prediction
        
        # Make prediction 
        for image in in_data:
            # Inference
            tic = time()
            out_data = ktc.kneron_inference([image], nef_file=args.nef, radix=7 , model=str(args.model), threads=args.threads)
            print(f'inference time {(time()-tic):.2f}')
            
            # Postprocess
            tic = time()
            det_res = postprocess(out_data, [image_shape[count][1], image_shape[count][0]], args.conf_t, args.iou_t)
            print(f'postprocess time {(time()-tic):.2f}') 
            
            # Write predictions in YOLO style
            predictions = [] # create tuple to fill with predictions
            for pred in range(len(det_res[2])):
                # Convert y1x1y2x2 to XcYcwh with relative values
                prediction = (str(det_res[2][pred]) + " " + str(det_res[1][pred]) + " " 
                    + str((det_res[0][pred][1] + det_res[0][pred][3])/2/input_image.size[0])+ " " 
                    + str((det_res[0][pred][0] + det_res[0][pred][2])/2/input_image.size[1])+ " " 
                    + str((det_res[0][pred][3] - det_res[0][pred][1])/input_image.size[0])+ " " 
                    + str((det_res[0][pred][2] - det_res[0][pred][0])/input_image.size[1]))

                predictions.append(prediction)
                
            predictions = '\n'.join(predictions)

            # Save predictions to directory args.path/predictions/*.txt
            with open(predictions_folder + "/" + images[count+args.model*step][:12] + ".txt", 'w') as f:
                f.write(str(predictions))

            count += 1
          
    
if __name__ == "__main__":
    main()

Writing /data1/parallel.py


## Download COCO dataset

In [11]:
!mkdir /workspace/COCO 
%cd /workspace/COCO

/workspace/COCO


In [12]:
!wget -c http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip
!rm val2017.zip


--2021-08-23 12:01:35--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.135.17
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.135.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  6.45MB/s    in 2m 4s   

2021-08-23 12:03:40 (6.27 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [13]:
!mv val2017 images 
!mkdir val2017
!mv images val2017/images
!mkdir /workspace/COCO/val2017/predictions

# Yolov3 

In [14]:
!python /data1/parallel.py -h

/workspace/miniconda/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
Using TensorFlow backend.
usage: parallel.py [-h] [--demo] [--image IMAGE] [--path PATH] [--nef NEF]
                   [--step STEP] [--init INIT] [--model MODEL]
                   [--threads THREADS] [--img-size IMG_SIZE] [--conf-t CONF_T]
                   [--iou-t IOU_T]

Runs an inference on multiple images.

optional arguments:
  -h, --help           show this help message and exit
  --demo               run demo on your image
  --image IMAGE        path to your image for demo
  --path PATH          directory of your images
  --nef NEF            path to your nef model
  -

In [ ]:
"""
This script will run untill no more predictions to be done.
We recommend to increase number of models, if you're using it to simulate Kneron 720, 
since this simulator uses only one thread.
"""

import multiprocessing
import os
from time import time
from os import listdir
from os.path import isfile, join
tic = time() 
step = 5 # images per iteration of script
models = 5 # number of models running in parallel
init = 3 # time between model's initialization
threads = 16 # number of workers for each model, works only for 520 model
path = '/workspace/COCO/val2017' # path to validation directory
predictions = path + '/' + 'predictions'
labels= path + '/' +  'images'


def missed(predictions, labels):    
    predictions =[f[:12] for f in listdir(predictions) if isfile(join(predictions, f))]    
    labels =[f[:12] for f in listdir(labels) if isfile(join(labels, f))]
    predicted = list(set(predictions) & set(labels))
    not_predicted = labels
    
    for element in predicted:
        #print(element)
        not_predicted.remove(element) 

    return not_predicted

# Check if there is all predictions
not_predicted = missed(predictions, labels)
all_processes =[]
for i in range(models):
    all_processes.append('/data1/parallel.py --path {} --model {} --step {} --init {} --conf-t 0.001 --threads {}'.format(path, i, step, init, threads))

while len(not_predicted):
    # This block of code enables us to call the script from command line.                                                                                
    def execute(process):                                                             
        os.system(f'python {process}')                                       


    process_pool = multiprocessing.Pool(processes = models)                                                        
    process_pool.map(execute, all_processes)
    
    # Check if there is all predictions
    not_predicted = missed(predictions, labels) 
    
print("Script ran for ", time() - tic)

# mAP evaluation

### Download COCO anotations

In [16]:
%cd /workspace/COCO/val2017
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget https://raw.githubusercontent.com/matlab-deep-learning/Object-Detection-Using-Pretrained-YOLO-v2/main/%2Bhelper/coco-classes.txt
!unzip annotations_trainval2017.zip
!mkdir labels

/workspace/COCO/val2017
--2021-08-23 12:11:47--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.1.20
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.1.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  9.34MB/s    in 41s     

2021-08-23 12:12:28 (5.94 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

--2021-08-23 12:12:29--  https://raw.githubusercontent.com/matlab-deep-learning/Object-Detection-Using-Pretrained-YOLO-v2/main/%2Bhelper/coco-classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting 

In [17]:
import json
f=open("annotations/instances_val2017.json")
coco= json.load(f)

In [18]:
coco.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [19]:
# COCO classes for inference
namescoco = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [20]:
# 91 classes from COCO's paper
names91 = ['empty', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
           'traffic light', 'fire hydrant', 'empty', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
           'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'empty', 'backpack', 
           'umbrella', 'empty', 'empty', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 
           'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 
           'bottle', 'empty', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
           'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
           'potted plant', 'bed', 'empty', 'dining table', 'empty', 'empty', 'toilet', 'empty', 'tv', 'laptop', 
           'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 
           'empty', 'book','clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

### Generate labels in yolo.txt format

In [21]:
for i in range(len(coco['images'])):
    bbox = []
    for j in range(len(coco['annotations'])):
        if coco['annotations'][j]["image_id"] == coco['images'][i]["id"]: 
            x = coco['annotations'][j]["bbox"][0]
            y = coco['annotations'][j]["bbox"][1]
            w =  coco['annotations'][j]["bbox"][2]
            h = coco['annotations'][j]["bbox"][3]
            width = coco['images'][i]['width']
            height = coco['images'][i]['height']
            # Convert class id to 80 classes version
            cat = coco['annotations'][j]['category_id']
            cls = names91[cat]
            cls = namescoco.index(cls)
            bbox.append(str(cls) + " " + str((x+w/2)/width) + " " + str((y+h/2)/height)+" " + str(w/width)+" " + str(h/height)) 
            
    bbox = '\n'.join(bbox)
    with open('/workspace/COCO/val2017/labels/{}.txt'.format(coco['images'][i]["file_name"][:12]), 'w') as f:
        f.write(str(bbox))

In [ ]:
# Evaluate predictions on these labels with https://github.com/rafaelpadilla/review_object_detection_metrics
# Download all needed files that located under /workspace/COCO/val2017 and use it to calculate mAP

In [ ]:
# Your results should look like this:
"""
COCO METRICS:
AP: 0.30763665649871075
AP50: 0.5915429205728568
AP75: 0.29242076553561813
APsmall: 0.10837458746989537
APmedium: 0.29600038344642876
APlarge: 0.4424104857026416
AR1: 0.2567070573097393
AR10: 0.4102890791340148
AR100: 0.4285984240348608
ARsmall: 0.23691939502550804
ARmedium: 0.44522361970282825
ARlarge: 0.5428652114891923
"""